# Run WFlow upper Zambezi and see if we can add water to groundwater #

- run model as is
- run model, and in every step, update zi with additional water
- plot


In [ ]:
%matplotlib inline
from bmi.wrapper import BMIWrapper
import matplotlib.pyplot as plt
import numpy as np
import wflow.wflow_bmi as wfbmi
import xarray as xr


1. This step quoted, skip to the sensitivity test below. (Run 4km model for a couple of time steps, extract a spatial end result, plot it and close (finalize) model.)

In [ ]:
# ini_file = r'/home/hcwinsemius/git/barotse/wflow/wflow_sbm_sensitivity.ini'

# bmimodel = wfbmi.wflowbmi_csdms()
# bmimodel.initialize_config(ini_file)
# bmimodel.initialize_model()

# for n in range(100):
#     print(n)
#     bmimodel.update()

# # finalize model
# bmimodel.finalize()


In [ ]:
ini_file = r'/home/hcwinsemius/git/barotse/wflow/wflow_sbm_sensitivity.ini'

bmimodel = wfbmi.wflowbmi_csdms()
bmimodel.initialize_config(ini_file)
bmimodel.initialize_model()

### Run 4km model again but now add a lot of water to the saturated store

In [ ]:
# we'll add water over a square box approximately covering Barotse
xmin, xmax, ymin, ymax = 130, 150, 80, 125
for n in range(100):
    print(n)
    bmimodel.update()
    zi = bmimodel.get_value("zi")
    zi[ymin:ymax, xmin:xmax] = np.maximum(0, zi[ymin:ymax, xmin:xmax] - 100)
    bmimodel.set_value("zi", zi)

# finalize model
bmimodel.finalize()


In [ ]:
zi = bmimodel.get_value("zi")
zi[ymin:ymax, xmin:xmax] = np.maximum(0, zi[ymin:ymax, xmin:xmax] - 2000)
plt.imshow(zi, vmin=0, vmax=3000, origin='lower')
plt.colorbar()
bmimodel.set_value("zi", zi)

### Plot zi to see if it is persistent

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import animation
step = 5 # one frame every <step> hr
# bmimodel.finalize()
def update_plot(i, da_fld, cax_fld):
    da0_fld = da_fld.isel(time=i)
    t = da0_fld.time.dt.strftime("%d-%B-%Y %H:%M:%S").item()
    ax.set_title(f'Wflow water depth under surface {t}')
    cax_fld.set_array(da0_fld.values)

fn = "/home/hcwinsemius/git/barotse/wflow/run_default/outmaps.nc"
ds = xr.open_dataset(fn)
zi = ds["zi"]
fig = plt.figure(figsize=(13, 10))
ax = plt.subplot(111)
cax_zi = zi.isel(time=0).plot(ax=ax, vmin=0, vmax=6000, cmap=plt.cm.Blues_r, cbar_kwargs=dict(aspect=30, fraction=.1))

ani = animation.FuncAnimation(
    fig,
    update_plot,
    frames=np.arange(0, 100, step),
    interval=250,     # ms between frames
    fargs=(zi, cax_zi)
)
from IPython.display import HTML
HTML(ani.to_html5_video())
# ani.save('zi.mp4', fps=4, dpi=200)